<a href="https://colab.research.google.com/github/paulaleite/aprendizagemDeMaquinaa2022/blob/main/Trabalho_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final para Disciplina de Aprendizagem de Máquina
### Paula Torales Leite
Textinho explicando o problema

## Importações

In [20]:
# Processamento de Dados
import pandas as pd
import numpy as np
import scipy.stats

# Similaridade
from sklearn.metrics.pairwise import cosine_similarity
import operator

## Preparação dos Dados
União dos dados da base de avaliações e de desafios, para que seja possível construir a base que será utilizada durante o processo.

In [4]:
# Leitura de dadas
avaliacoes = pd.read_csv('./dados/avaliacoes.csv', sep=';')
avaliacoes.head()

,userID,challengeID,objectiveID,rating
0,24,4,30,2
1,8,6,25,3
2,7,3,31,2
3,30,4,5,4
4,27,7,49,1


In [5]:
# Informação sobre os dados
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   userID       833 non-null    int64
 1   challengeID  833 non-null    int64
 2   objectiveID  833 non-null    int64
 3   rating       833 non-null    int64
dtypes: int64(4)
memory usage: 26.2 KB


In [6]:
print('O dataset de avaliações tem', avaliacoes['userID'].nunique(), 'usuários únicos')
print('O dataset de avaliações tem', avaliacoes['challengeID'].nunique(), 'desafios únicos')
print('O dataset de avaliações tem', avaliacoes['objectiveID'].nunique(), 'objetivos únicos')
print('O dataset de avaliações tem', avaliacoes['rating'].nunique(), 'unique ratings')
print('As avalições únicas são', sorted(avaliacoes['rating'].unique()))

O dataset de avaliações tem 30 usuários únicos
O dataset de avaliações tem 8 desafios únicos
O dataset de avaliações tem 55 objetivos únicos
O dataset de avaliações tem 5 unique ratings
As avalições únicas são [1, 2, 3, 4, 5]


In [7]:
objetivos = pd.read_csv('./dados/objetivos.csv', sep=';')
objetivos.head()

,objectiveID,title
0,1,Objetivo1
1,2,Objetivo2
2,3,Objetivo3
3,4,Objetivo4
4,5,Objetivo5


In [8]:
# Merge das avaliações e objetivos
df = pd.merge(avaliacoes, objetivos, on='objectiveID', how='inner')
df.head()

,userID,challengeID,objectiveID,rating,title
0,24,4,30,2,Objetivo30
1,26,4,30,5,Objetivo30
2,12,2,30,2,Objetivo30
3,17,6,30,3,Objetivo30
4,12,2,30,1,Objetivo30


## Análise Exploratória
Remoção dos dados duplicados, bem como a escolha do Challenge que será focado.

In [9]:
df_duplicados = df.copy()

In [10]:
df_duplicados.duplicated().sum()

5

In [46]:
df_duplicados = df_duplicados.drop_duplicates()

In [14]:
df_duplicados.duplicated().sum()

0

In [143]:
desafio_atual = 3

df_filtrado = df_duplicados[df_duplicados.challengeID == desafio_atual]
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 12 to 829
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       114 non-null    int64 
 1   challengeID  114 non-null    int64 
 2   objectiveID  114 non-null    int64 
 3   rating       114 non-null    int64 
 4   title        114 non-null    object
dtypes: int64(4), object(1)
memory usage: 5.3+ KB


## Criando a Matriz
Criação da Matriz advinda do dataset de base.

In [145]:
matriz = df_filtrado.pivot_table(index='title', columns='userID', values='rating')
matriz.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Normalização dos Dados
Normalização dos dados da matriz.

In [146]:
matriz_norm = matriz.subtract(matriz.mean(axis=1), axis = 0)
matriz_norm.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,-2.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,NaN,NaN,1.333333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Score de Similaridade
Construção da similiaridade por item utilizando a correlação de Pearson

In [147]:
similiaridade_de_item = matriz_norm.T.corr()
similiaridade_de_item.head()

title,Objetivo10,Objetivo11,Objetivo12,Objetivo13,Objetivo15,Objetivo16,Objetivo17,Objetivo19,Objetivo2,Objetivo21,...,Objetivo50,Objetivo51,Objetivo52,Objetivo53,Objetivo54,Objetivo55,Objetivo6,Objetivo7,Objetivo8,Objetivo9
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo12,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Função de Recomendação de Objetivos

In [148]:
def rec_baseada_item(userID_escolhido, numero_itens_similares, numero_recomendacoes):

  # Objetivos que o usuário não avaliou
  userID_escolhido_sem_avaliacao = pd.DataFrame(matriz_norm[userID_escolhido].isna()).reset_index()
  userID_escolhido_sem_avaliacao = userID_escolhido_sem_avaliacao[userID_escolhido_sem_avaliacao[userID_escolhido]==True]['title'].values.tolist()

  # Objetivos que o usuário avaliou
  userID_escolhido_avaliado = pd.DataFrame(matriz_norm[userID_escolhido].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={userID_escolhido:'rating'})
  
  # Dicionário para salvar os objetivos não avaliados e o resultado da predição
  avaliacao_predicao = {}
         
  for objetivo_atual in userID_escolhido_sem_avaliacao: 
    # Cálcula da similaridade do objetivo atual com outros filmes
    similaridade_objetivo_atual = similiaridade_de_item[[objetivo_atual]].reset_index().rename(columns={objetivo_atual:'score_similaridade'})

    # Rank das similaridades entre os objetivos do usuário selecionado e o objetivo atual
    userID_escolhido_avaliado_similarity = pd.merge(left=userID_escolhido_avaliado, 
                                                right=similaridade_objetivo_atual, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('score_similaridade', ascending=False)[:numero_itens_similares]

    # Cálculo da classificação prevista usando a média ponderada das pontuações de similaridade e as classificações do usuário 1
    predicao_avaliacao = round(np.average(userID_escolhido_avaliado_similarity['rating'], 
                                        weights=userID_escolhido_avaliado_similarity['score_similaridade']), 6)
    
    # Salvando a avaliação prevista no dicionário, apenas no caso em que não exista NaN
    if pd.isna(predicao_avaliacao) == False:
      avaliacao_predicao[objetivo_atual] = predicao_avaliacao

  # Retorno das avaliações principais
  return sorted(avaliacao_predicao.items(), key=operator.itemgetter(1), reverse=True)[:numero_recomendacoes]

In [161]:
objetivos_recomendados = rec_baseada_item(userID_escolhido = 4, numero_itens_similares = 1, numero_recomendacoes = 3)
objetivos_recomendados

[('Objetivo16', 0.666667), ('Objetivo12', -0.333333)]